In [14]:
import numpy as np
from deepsy.core.model import Model
from deepsy.core.nn import NeuralNetwork
from deepsy.core.layer import Layer
from deepsy.core.functions.activation_functions import *
from deepsy.core.functions.loss_functions import *
from deepsy.core.functions.optimizers import *
import pandas as pd
from sklearn.model_selection import train_test_split

# 1) titanic.csv

### Data:

In [34]:
dataset = pd.read_csv('data/binary/titanic.csv')
Y = dataset['Survived']
X = dataset.drop(columns=['Survived'])
X = X.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
X = pd.get_dummies(X, columns=['Pclass', 'Sex', 'Embarked'])

X['Age'].fillna(X['Age'].mean(), inplace=True)
X['Fare'].fillna(X['Fare'].mean(), inplace=True)

X = (X - X.mean()) / X.std()

X, Y = X.values, Y.values

print('Dataset shape: {}'.format(dataset.shape))
print(dataset.head())
print()
print(X[:5])
print()
print(Y[:5])
print()

Dataset shape: (891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8

### Training:

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

nn = NeuralNetwork(layers=[
    Layer(nr_neurons=12, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=16, nr_in_features=12, activation_func=ReLU(), dropout_rate=0.15),
    Layer(nr_neurons=8, nr_in_features=16, activation_func=ReLU(), dropout_rate=0.15),
    Layer(nr_neurons=1, nr_in_features=8, activation_func=Sigmoid())
])

model = Model(nn, loss_func=BinaryCrossEntropy(), optimizer=GradientDescent(learning_rate=0.2))
model.train(X_train.T, Y_train.T, nr_epochs=400, reg_lambda=0.2)

Epoch 1: cost = 0.7315835607872729
Epoch 2: cost = 0.7346364843634535
Epoch 3: cost = 0.730866549247467
Epoch 4: cost = 0.7128417610858127
Epoch 5: cost = 0.7089894094210158
Epoch 6: cost = 0.7196835476982486
Epoch 7: cost = 0.7006294310031385
Epoch 8: cost = 0.7070372362966171
Epoch 9: cost = 0.7149159206187055
Epoch 10: cost = 0.7036270669616692
Epoch 11: cost = 0.7054023656833059
Epoch 12: cost = 0.6948303118898463
Epoch 13: cost = 0.7023103978400446
Epoch 14: cost = 0.6964733051417826
Epoch 15: cost = 0.6990097492409253
Epoch 16: cost = 0.6892307370175489
Epoch 17: cost = 0.6973147052282749
Epoch 18: cost = 0.6909242384707911
Epoch 19: cost = 0.6819836990109244
Epoch 20: cost = 0.6931454638377845
Epoch 21: cost = 0.6890183777368833
Epoch 22: cost = 0.6814308800886664
Epoch 23: cost = 0.6889575719927136
Epoch 24: cost = 0.6886654114070844
Epoch 25: cost = 0.6883035050515484
Epoch 26: cost = 0.6889116308204744
Epoch 27: cost = 0.6876766136217533
Epoch 28: cost = 0.6871804632287266
Ep

### Testing:

In [40]:
Y_predicted = model.predict(X_test.T)
acc = sum((Y_predicted.reshape(Y_test.shape) > 0.5) == Y_test) / len(Y_test)
print('Accuracy: {}'.format(acc))
for i in range(len(Y_predicted[0])):
    print('predicted: {}   actual: {}'.format((Y_predicted[0][i] > 0.5).astype(int), Y_test[i]))

Accuracy: 0.7932960893854749
predicted: 0   actual: 1
predicted: 0   actual: 1
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 1   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 1   actual: 1
predicted: 0   actual: 1
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 1   actual

### Parameters:

In [41]:
for layer in model.get_neural_network().get_layers():
    print(layer.get_parameters())

{'W': array([[-0.54498117,  0.02941081, -0.93485692, -0.34346945,  0.60608652,
         0.73841659,  0.11937626, -0.10676991,  0.39640713,  0.16729861,
        -0.50152983, -0.26863477],
       [ 0.38008134, -0.02012299,  0.11502053,  0.4021095 , -0.02705442,
        -0.3129756 ,  0.24432841,  0.05794498,  0.96060367,  0.45931097,
        -0.69118143,  1.26381282],
       [ 0.14043181, -0.02847278, -0.96390143,  0.25428025, -0.69154621,
         0.95668832, -0.03771398,  0.46663264,  0.11441669, -0.24377148,
        -0.59610401,  0.71752043],
       [-0.1772373 , -0.4020625 ,  0.07229826,  0.69099429,  0.83636497,
         0.10629988, -0.61137939,  0.05411629,  0.07613289, -0.43242972,
        -0.14694231, -0.43741037],
       [ 0.12978711, -0.10765753, -0.91692404,  0.05195995, -0.2983985 ,
         0.0514821 ,  0.80273987,  0.44065525, -0.18074015,  0.09725239,
        -0.52971259,  0.20977122],
       [-0.02062587,  0.08579149, -0.40060272, -0.19797855, -0.33617036,
         0.12404

# 2) Employee.csv

### Data:

In [43]:
dataset = pd.read_csv('data/binary/Employee.csv')
Y = dataset['LeaveOrNot']
X = dataset.drop(columns=['LeaveOrNot'])
X = pd.get_dummies(X, columns=['Education', 'City', 'Gender', 'EverBenched'])

X = (X - X.mean()) / X.std()

X, Y = X.values, Y.values

print('Dataset shape: {}'.format(dataset.shape))
print(dataset.head())
print()
print(X[:5])
print()
print(Y[:5])

Dataset shape: (4653, 9)
   Education  JoiningYear       City  PaymentTier  Age  Gender EverBenched  \
0  Bachelors         2017  Bangalore            3   34    Male          No   
1  Bachelors         2013       Pune            1   28  Female          No   
2  Bachelors         2014  New Delhi            3   38  Female          No   
3    Masters         2016  Bangalore            3   27    Male          No   
4    Masters         2017       Pune            3   24    Male         Yes   

   ExperienceInCurrentDomain  LeaveOrNot  
0                          0           0  
1                          3           1  
2                          2           0  
3                          5           1  
4                          2           1  

[[ 1.03952665  0.53744523  0.95454254 -1.86470098  0.54044268 -0.48052341
  -0.20000086  1.04316163 -0.57522042 -0.61197491 -0.82146269  0.82146269
   0.33832885 -0.33832885]
 [-1.10711376 -3.024852   -0.28870069  0.06054761  0.54044268 -0.4805234

### Training:

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

nn = NeuralNetwork(layers=[
    Layer(nr_neurons=12, nr_in_features=14, activation_func=ReLU()),
    Layer(nr_neurons=16, nr_in_features=12, activation_func=ReLU(), dropout_rate=0.3),
    Layer(nr_neurons=8, nr_in_features=16, activation_func=ReLU(), dropout_rate=0.3),
    Layer(nr_neurons=1, nr_in_features=8, activation_func=Sigmoid())
])

model = Model(nn, loss_func=BinaryCrossEntropy(), optimizer=GradientDescent(0.4))
model.train(X_train.T, Y_train.T, nr_epochs=500, reg_lambda=0.3)

Epoch 1: cost = 0.9662521704853717
Epoch 2: cost = 0.9002246094400375
Epoch 3: cost = 0.8387590938696978
Epoch 4: cost = 0.8016330983534719
Epoch 5: cost = 0.7650810823968069
Epoch 6: cost = 0.7439934945048581
Epoch 7: cost = 0.7188157328424173
Epoch 8: cost = 0.7099660663228402
Epoch 9: cost = 0.6944740926679744
Epoch 10: cost = 0.6829052689029167
Epoch 11: cost = 0.6743591328499103
Epoch 12: cost = 0.6676899297305788
Epoch 13: cost = 0.6665966269701958
Epoch 14: cost = 0.6628385555654904
Epoch 15: cost = 0.655886414889925
Epoch 16: cost = 0.6563547252033289
Epoch 17: cost = 0.6492915720847161
Epoch 18: cost = 0.6468859394915096
Epoch 19: cost = 0.6464360663841505
Epoch 20: cost = 0.6456907751851779
Epoch 21: cost = 0.6402460513877384
Epoch 22: cost = 0.6362363466275228
Epoch 23: cost = 0.6380442152962964
Epoch 24: cost = 0.6420816878245487
Epoch 25: cost = 0.6319847951901558
Epoch 26: cost = 0.6342126088062405
Epoch 27: cost = 0.6310047203310775
Epoch 28: cost = 0.6324901037530631
Ep

### Testing:

In [47]:
Y_predicted = model.predict(X_test.T)
acc = sum((Y_predicted.reshape(Y_test.shape) > 0.5) == Y_test) / len(Y_test)
print('Accuracy: {}'.format(acc))
for i in range(len(Y_predicted[0])):
    print('predicted: {}   actual: {}'.format((Y_predicted[0][i] > 0.5).astype(int), Y_test[i]))

Accuracy: 0.7980665950590763
predicted: 0   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 1   actual: 1
predicted: 0   actual: 1
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 1   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 1   actual: 1
predicted: 1   actual: 1
predicted: 1   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 1
predicted: 0   actual: 0
predicted: 1   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual: 0
predicted: 0   actual

In [48]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, (Y_predicted.reshape(Y_test.shape) > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       603
           1       0.82      0.54      0.65       328

    accuracy                           0.80       931
   macro avg       0.81      0.74      0.76       931
weighted avg       0.80      0.80      0.79       931

